# Programming Assignment 3 (PA3): Least squares & seamless copy/paste

### CS70 — Foundations of Applied Computer Science
---

<div class="alert alert-warning">
</div>

## Import statements

<div class="alert alert-info" role="alert">
    <b>Run</b> the following cell in the Jupyter Notebook to include the required modules. It imports libraries such as NumPy and Matplotlib and configures them to produce nice graphics.
</div>

In [1]:
import numpy as np  # for building and manipulating matrices
import time  # for measuring time elapsed while running code

# for sparse matrix operations
from scipy.sparse import lil_matrix
import scipy.sparse.linalg as sla
from skimage.transform import resize

# for graphics
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
# for interactive graph features
%matplotlib notebook  
%config InlineBackend.figure_format = 'retina'  # nice retina graphics


## Background
The primary goal of this assignment is to seamlessly blend an object or texture from a source image into a target image. The simplest method would be to just copy and paste the pixel values from one image directly into the other. Unfortunately, this will create very noticeable seams, even if the backgrounds are well-matched. How can we get rid of these seams without doing too much perceptual damage to the source region?

![](images/setup.png)

The insight is that our visual system is often more sensitive to the gradient of an image (how quickly the pixel values change left-right and up-down) than the overall intensity. We will therefore set up the problem as finding values for the pasted target pixels that maximally preserve the gradient of the source region without changing any of the background pixels. Note that we are making a deliberate decision here to ignore the overall intensity! So a green hat could turn red, but it will still look like a hat.

<div class="alert alert-info" role="alert">
<b>Implement the Least-Squares Solver</b>

Implement `least_squares_2D`, which takes two parameters, `im_s` and `im_background`, each of which have dimensions $(W + 2) \times (H + 2)$. The function returns the interior region (`im_blend`, dimensions $W \times H$) of `im_s` and at the same time paste `im_blend` onto the interior region of `im_background`. Your tasks include the following:

<ol>
<li>Calculate the dimensions of `A` and `b`.</li>

<li>Create a matrix `im2var` to map each pixel of interest to a variable index (vectorization). </li>

<li>Initialize entries in `A` and `b`.</li>

<li>Solve the least-squares problem and save the results in `v`.</li>
    
<li>Assign the solved values to `im_blend`.</li>
</ol>

The starter code will return the resultant images as a `im_blend`. 
</div>

In [2]:
def least_squares_2D(im_s, im_background):
    #### initialize results to just be copies of the background
    im_blend = im_background.copy()
    
    #### initialize the size of source image 
    imh, imw = (im_s.shape[0], im_s.shape[1])    #### imh: image hight, imw: image width
    imd = 1 if im_s.ndim < 3 else im_s.shape[2]  #### number of components of each pixel, e.g., for an RGB image, imd=3
    

    num_vars = (imw - 2) * (imh - 2)
    num_eqns = ((imh - 1) * (imw - 2)) + ((imh - 2) * (imw - 1))
    
    
    # Create an im2var matrix with the same dimensions as im_s
    # and assign coordinates inside the 1 pixel padding unique numbers for easy indexing.
    im2var = np.zeros((imh - 2, imw - 2), dtype = int)
    ct = 0
    for x in range(imh - 2):
            for y in range(imw - 2):
                im2var[x][y] = ct
                ct += 1
    
    #### check if the mapping is initialized correctly
    print(im2var)
    
    #### initialize A and b ####
    A = lil_matrix((num_eqns, num_vars))
    b = np.zeros((num_eqns, imd))
    
    #### Fill the elements of A and b ####
    idx = 0
    for x in range(imh - 1):
        for y in range(imw - 2):
            if x == 0:
                A[idx, im2var[x][y]] = 1
                b[idx] = im_background[x][y] - im_s[x][y+1] + im_s[x+1][y+1]
            elif x == imh - 2:
                A[idx, im2var[x-1][y]] = -1
                b[idx] = im_s[x+1][y + 1] - im_s[x][y+1] - im_background[x+1][y+1]
            else:
                A[idx, im2var[x][y]] = 1
                A[idx, im2var[x-1][y]] = -1
                b[idx] = im_s[x+1][y+1] - im_s[x][y+1] 
            idx += 1
            
    for x in range(imh - 2):
        for y in range(imw - 1):
            if y == 0:
                A[idx, im2var[x][y]] = 1
                b[idx] = im_background[x][y] - im_s[x+1][y] + im_s[x+1][y+1]
            elif y == imw - 2:
                A[idx, im2var[x][y-1]] = -1
                b[idx] = im_s[x + 1][y + 1] - im_s[x+1][y] - im_background[x+1][y+1]
            else:
                A[idx, im2var[x][y]] = 1
                A[idx, im2var[x][y-1]] = -1
                b[idx] = im_s[x+1][y+1] - im_s[x+1][y]
            idx += 1
            
    
    #### solve the least-squares problem
    
    #### convert A to (CSR) sparse format
    A = A.tocsr()
    
    print('Solving sparse system using sla.lsqr...')
    t = time.time()

    #### solve for all channels
    v = np.zeros((num_vars, imd))
    for c in range(0,imd):
        #### Solve the least-squares problem Av=b, with the right-hand side specified by the c'th column of b ####
        
        v[:, c] = sla.lsqr(A, b[:, c])[0]
        
        
        #### copy over to im_blend
        if im_s.ndim > 2: #### this branch is for rgb image
            #### Reshape v to an appropriate shape and copy its values to the c'th component of im_blend ####
            
            im_blend[1:imh - 1, 1:imw - 1, c] = v[:, c].reshape(imh - 2, imw - 2)
            
            
        else:  #### this branch is for gray-scale image     
            #### Reshape v to an appropriate shape and copy its values to im_blend ####
            
            im_blend[1:imh - 1, 1:imw - 1] = v[:, 0].reshape(imh - 2, imw - 2)
            
    
    elapsed = time.time() - t
    print("\tTotal time for sparse solve: {} seconds \n".format(elapsed))

    #### return the blended result
    return im_blend

### Testing `least_squares_2D`

Test Case 1: Before testing `least_squares_2D` on a larger image, test it on the 4 x 4 `source_im` and `background_im` from the simple 2D section in the code block below. Display the source image, background image, the output using `sla.lsqr`, the output using the normal equations, and the expected result. **If implemented correctly, the output should be the exact same as the output expected in the simple 2D section.**

[[0 1]
 [2 3]]
Solving sparse system using sla.lsqr...
	Total time for sparse solve: 0.00028014183044433594 seconds 



<IPython.core.display.Javascript object>


In [3]:
expected = np.array([[ 10,  10,  10,  10],
                     [ 10, 164, 110,  10],
                     [ 10,  20,  30,  10],
                     [ 10,  10,  10,  10]])

source_im = np.array([[100, 100, 100, 100],
                          [100, 255, 200, 100],
                          [100, 110, 120, 100],
                          [100, 100, 100, 100]])
background_im = np.array([[10, 10, 10, 10],
                          [10, 10, 10, 10],
                          [10, 10, 10, 10],
                          [10, 10, 10, 10]])

# call least_squares_2D on source_im, background_im
# display source, target, sla.lsqr output, normal equation output, expected output

# simple paste of 2x2 square onto 4x4 square
size = 4
im_out = least_squares_2D(source_im, background_im)

fig, axs = plt.subplots(1,4)
fig.set_size_inches(8,5)
fig.tight_layout(pad=0.0)

#### draw source_im
axs[0].imshow(source_im, cmap='gray', vmin=0, vmax=255)
axs[0].set_title('Source', fontsize=8)
axs[0].set_yticks(np.arange(0, size))

#### draw background_im
axs[1].imshow(background_im, cmap='gray', vmin=0, vmax=255)
axs[1].set_title('Target', fontsize=8)
axs[1].set_yticks(np.arange(0, size))

#### draw your result after calling least_squares_2D
axs[2].imshow(im_out, cmap='gray', vmin=0, vmax=255)
axs[2].set_title('Result - Least Squares', fontsize=8)
axs[2].set_yticks(np.arange(0, size))

#### draw the expected result
axs[3].imshow(expected, cmap='gray', vmin=0, vmax=255)
axs[3].set_title('Expected Result', fontsize=8)
axs[3].set_yticks(np.arange(0, size))

plt.show()

Test Case 2: Now load `toyim = plt.imread('images/toy_problem.png')`. Create `im_background` by creating a matrix with the same dimensions as `toyim` filled with the value at the top left corner of `toyim` using np.full [see numpy.full documentation](https://numpy.org/doc/stable/reference/generated/numpy.full.html). We will test `least_squares_2D` on a larger image by reconstructing `toyim`. Run `least_squares_2D` on `toyim` and `im_background`, and display the two reconstructed images with titles.
**If you implemented everything correctly, you should see the same original and reconstructed images and the error value should be a small number.**

[[    0     1     2 ...   105   106   107]
 [  108   109   110 ...   213   214   215]
 [  216   217   218 ...   321   322   323]
 ...
 [12312 12313 12314 ... 12417 12418 12419]
 [12420 12421 12422 ... 12525 12526 12527]
 [12528 12529 12530 ... 12633 12634 12635]]
Solving sparse system using sla.lsqr...
	Total time for sparse solve: 0.07134580612182617 seconds 



<IPython.core.display.Javascript object>


Error: 2.638464259955608e-05


In [4]:
toyim = plt.imread('images/toy_problem.png')
im_background = np.full(toyim.shape, toyim[0, 0])

im_out = least_squares_2D(toyim, im_background)

# show the original image
fig2, axs2 = plt.subplots(1,2)
fig2.tight_layout(pad=1.0)
fig2.set_size_inches(8,4)

axs2[0].imshow(toyim, cmap = plt.cm.gray)
axs2[0].set_title("Original image")
    
# show the reconstruction
axs2[1].imshow(im_out, cmap = plt.cm.gray)
axs2[1].set_title("Reconstructed image")  

print('Error: {error}'.format(error=np.linalg.norm(im_out - toyim)/(toyim.size)))
plt.show()

## Poisson Blending

For Poisson Blending, select a coordinate on the background image which will serve as the bottom right corner for an object image to be blended into the background image. Ideally, the background of the object image and the surrounding area of the target region will be of similar color. 

**Select the coordinates for the gradient blend by clicking on the interactive figure. The interactive figure should draw a rectangle where the object image will appear in the blended image.**

<IPython.core.display.Javascript object>


In [5]:
## new gradient domain copy-paste code should go here
# read in background and object images
im_background = plt.imread('images/background.jpg')/255.0
im_object = plt.imread('images/penguin-chick.jpg')/255.0
#
# reduce the size of the image for faster performance while debugging
# You can comment these two lines out to use the full-resolution images
im_background = resize(im_background, (im_background.shape[0] // 5,
                                       im_background.shape[1] // 5),
                       anti_aliasing=True)
im_object = resize(im_object, (im_object.shape[0] // 5,
                               im_object.shape[1] // 5),
                       anti_aliasing=True)

# get source region mask from the user
objh, objw, _ = im_object.shape
objmask = np.ones((objh, objw))

# for storing the selected coordinates aligning blending
coords = np.zeros(2)

# handling coordinate selection by storing selected coordinates
def onclick(event):
    global coords, objh, objw
    coords = np.round([event.xdata, event.ydata]).astype(np.int)
    plt.title(f"Background Image (Selected Coordinate {coords[0]}, {coords[1]})")
    # draw rectangle on image
    rect = Rectangle((event.xdata-objw,event.ydata-objh),objw,objh,linewidth=1,edgecolor='r',facecolor='none')
    plt.gca().add_patch(rect)

# display interactive figure for selecting coordinate in background image
fig = plt.figure()
fig.set_size_inches(10,8)
plt.imshow(im_background)
cid = fig.canvas.mpl_connect('button_press_event', onclick) # for handling button click
plt.title("Background Image (Click a place to blend object image)")
plt.show()


<div class="alert alert-info" role="alert">
<b>Implement the Copy Copy-Paste Operation</b>
    
`simple_copy_paste`  simply copies the pixel values from `im_object` onto the appropriate location of `im_background`.

 `poisson_copy_paste` takes three parameters: `coords`, the coordinates of the bottom right corner of the region to blend, `im_object`, the image to paste into the background image, and `im_background`, the unaltered background image. You already cut out an appropriate region of `im_background` in the previous step. **(1) solve the `im_blend` for the blended region using the `least_squares_2D` you implemented and (2) paste the results back into the appropriate region of a copy of `im_background` (named `res` in the code).** 
</div>

In [6]:
coords = (150, 200)

def simple_copy_paste(coords, im_object, im_background):
    x, y = coords
    objh, objw, _ = im_object.shape
    
    # paste pixel values into im_background
    result = im_background.copy()
    result[y-objh:y, x-objw:x, :] = im_object
    
    return result

def poisson_copy_paste(coords, im_object, im_background):
    x, y = coords
    objh, objw, _ = im_object.shape

    res = im_background.copy()
    background = im_background[y-objh:y, x-objw:x, :]
    
    #### cut out the appropriate region of im_background and paste back into im_background ####
    res[y-objh:y, x-objw:x, :] = least_squares_2D(im_object, background)
    
   
    
    return res

<div class="alert alert-info" role="alert">
    Run `poisson_copy_paste` and display your blended image. If implemented correctly, print statements from the code should appear detailing the progress and time needed to execute least squares. You might need to restrict the blended image values to between 0 and 1 by calling `.clip(0,1)` on your blended images.
</div>

<IPython.core.display.Javascript object>


[[   0    1    2 ...   55   56   57]
 [  58   59   60 ...  113  114  115]
 [ 116  117  118 ...  171  172  173]
 ...
 [4060 4061 4062 ... 4115 4116 4117]
 [4118 4119 4120 ... 4173 4174 4175]
 [4176 4177 4178 ... 4231 4232 4233]]
Solving sparse system using sla.lsqr...
	Total time for sparse solve: 0.05797123908996582 seconds 



In [7]:
res0 = simple_copy_paste(coords, im_object, im_background)

fig3, axs3 = plt.subplots(1,3)
fig3.tight_layout(pad=0.0)
fig3.set_size_inches(18,5)

# show simple copy-paste first
axs3[0].imshow(res0.clip(0, 1))
axs3[0].set_title("Simple")


# add two more sub-plots with your poisson copy-paste results
res1 = poisson_copy_paste(coords, im_object, im_background)
axs3[1].imshow(res1.clip(0, 1))
axs3[1].set_title("Poisson")

plt.show()